In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict

args = read_args(api_key='../keys/production_api_scicrunch_key.txt', db_url='../keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(engine_key=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, engine_key=args.db_url)

In [ ]:
terms = sql.get_terms()

In [ ]:
terms

In [ ]:
superclasses = sql.get_superclasses()

In [ ]:
#term_superclasses
# id         tid1      tid2
#[(12461, 15030, 160, 1, 1491829905),
# (12462, 15031, 160, 1, 1491830438),
# (12463, 15032, 5022, 1, 1491831300),
# (147945, 150526, 0, 1, 1515533600),
# (147946, 150527, 0, 1, 1515533600),
# (147951, 150532, 0, 1, 1515534044),
# (147956, 150537, 0, 1, 1515534088),
# (147957, 150538, 0, 1, 1515534088)]

In [ ]:
terms.loc[terms.ilx == 'ilx_0232950']

In [ ]:
terms.loc[terms.label == 'src_subject_id (Minimally Verbal - Common Demographic Intake Form)']

In [ ]:
superclasses.loc[superclasses.tid == 134260]

In [ ]:
superclasses.loc[superclasses.id == 0]

In [ ]:
superclass_dict = {}
for i, row in superclasses.iterrows():
    superclass_dict.update({str(row.id):row})
    if int(row.superclass_tid) == 0:
        print(i)

In [ ]:
360 + 80*2 + 190*2 + 160*2 + 200 + 200 + 100 + 100 + 190

In [ ]:
1900 + 150 + 400

In [ ]:
for i, row in terms.iterrows():
    try:
        if not superclass_dict.get(str(row['id'])).empty:
            if str(row['type']) == 'cde':
                print(row); break
    except:
        pass

In [ ]:
tier1 = '/home/troy/Desktop/NDA/nda-cdes-first-tier-only-update-ready.json'
tier2 = '/home/troy/Desktop/NDA/complete_nda_tier2_production_ref.json'
nda_names = list(json.load(open(tier1)).keys()) + list(json.load(open(tier2)).keys())
tier1_data = json.load(open(tier1))
tier2_data = json.load(open(tier2))

In [ ]:
labels_to_id = sql.get_labels_to_ids_dict()
terms_df = sql.get_terms()

## Checking if any missing T1 labels

In [ ]:
t1_names = [name.lower().replace("'", '').strip() for name in list(tier2_data)] #just ended up getting rid of the ' altogether

for name in t1_names:
    if not labels_to_id.get(name):
        print(name)

In [ ]:
def link(ilxid):
    return "https://scicrunch.org/scicrunch/interlex/view/%s?searchTerm=%s"%(ilxid, ilxid)

In [ ]:
link(

In [ ]:
data = []
for name in t1_names:
    tid = labels_to_id[name]
    seg_df = terms_df.loc[terms_df.id == tid['cde']]
    if not seg_df.empty:
        if 'modifiedDate' in str(seg_df.definition.iloc[0]):
            print(name, tid)
            print(link(seg_df['ilx'].iloc[0]))
            data.append({'id':tid['cde'], 'definition':' '})
json.dump(data, open('../clean_t1_definitions.json', 'w'), indent=4)

In [ ]:
nda_ids = []
for label in nda_names:
    label = label.lower().strip().replace('"', '').replace("'", '')
    if not labels_to_id.get(label):
        sys.exit(label)
    else:
        nda_ids.append(labels_to_id[label]['cde'])

In [ ]:
terms_df = sql.get_terms()
annotations_df = sql.get_annotations()

In [ ]:
id_to_label = sql.get_ids_to_labels_dict()

In [ ]:
annotations_df.loc[annotations_df.item_id == 230000].to_dict('list')

In [ ]:
nda_ids[0]['cde']

In [ ]:
annotations_df.loc[annotations_df.label == 'eventname (ABCD Children&#39;s Report of Parental Behavioral Inventory)']

In [ ]:
%%time
cdes_no_annotations = []
for id in nda_ids[:]:
    hit = annotations_df.loc[annotations_df.tid == id]
    if hit.empty:
        cdes_no_annotations.append({'id':id, 'label':id_to_label[id]})
print(len(cdes_no_annotations))
#with open('/home/troy/Desktop/NDA/missing_annotaions.json', 'w') as f:
#json.dump(cdes_no_annotations, f, indent=4)

In [ ]:
for id in nda_ids:
    if id == 15813:
        print('yas')

In [ ]:
#%%time
cdes_incomplete_annotations = []
for id in nda_ids[:]:
    if id != 15813: continue
    print(id)
    hit = annotations_df.loc[annotations_df.tid == id]
    if hit.empty:
        pass#cdes_no_annotations.append({'id':id, 'label':id_to_label[id]})
    else:
        vs = hit.to_dict('list')['value']
        if '(abcd children' in id_to_label[id].lower():
            print(id_to_label[id])
            sys.exit(vs)
        #if 'id' not in ''.join(vs) and '(' in id_to_label[id] and id > 16000:
            #print(vs)
        #if len(hit.to_dict('list')['value']) not in [5, 8]:
        #    cdes_incomplete_annotations.append({'id':id, 'label':id_to_label[id]})
        #    sys.exit(cdes_incomplete_annotations)
hit           
#print(len(cdes_incomplete_annotations))
#with open('/home/troy/Desktop/NDA/incomplete_annotaions.json', 'w') as f:
#    json.dump(cdes_incomplete_annotations, f, indent=4)

In [ ]:
missing_annotatons_data = json.load(open('/home/troy/Desktop/NDA/missing_annotaions.json'))
incomplete_annotations_data = json.load(open('/home/troy/Desktop/NDA/incomplete_annotaions.json'))

In [ ]:
tier2_data

In [ ]:
''' Fix missing annotations '''
annotations = []
for e in missing_annotatons_data:
    if tier1_data.get(e['label']):
        t1 = tier1_data[e['label']]
        for v in t1['value']:
            annotations.append({'tid':e['id'], 'annotation_tid':15074, 'value':v})
    elif tier2_data.get(e['label']):
        t2 = tier2_data[e['label']]
        for v in t2['value']:
            annotations.append({'tid':e['id'], 'annotation_tid':15074, 'value':v})
    else:
        sys.exit(e)
len(annotations)
with open('/home/troy/Desktop/NDA/missing_annotaions_client_ready.json', 'w') as f:
    json.dump(annotations, f, indent=4)

In [ ]:
annotations = []
for e in incomplete_annotations_data:
    
    if tier1_data.get(e['label']):
        t1 = tier1_data[e['label']]
        for v in t1['value']:
            annotations.append({'tid':str(e['id']), 'annotation_tid':15074, 'value':str(v)})
    
    elif tier2_data.get(e['label']):
        t2 = tier2_data[e['label']]
        for v in t2['value']:
            annotations.append({'tid':str(e['id']), 'annotation_tid':15074, 'value':str(v)}) 
        print(e['label'])
    else:
        sys.exit(e)
        
len(annotations)
with open('/home/troy/Desktop/NDA/incomplete_annotaions_client_ready.json', 'w') as f:
    json.dump(annotations, f, indent=4)

In [ ]:
## So the shortNames list is the only complete data for some unknown reason at the moment; best to just use that to make pure annotations and update defintions
tier2 = json.load(open('/home/troy/Desktop/NDA/nda_shortName_data.json', 'r'))

In [ ]:
complete_nda_tier2_production_ref = defaultdict(dict)
for k, v in tier2.items():
    print(k)
    for d in v['dataElements']:
        print(d)
        annotations = []
        for annotation_key, annotations_values in v.items():
            if isinstance(annotations_values, list) and annotations_values:
                #print(annotation_key, annotations_values)
                value = annotation_key + ' : ' + '; '.join(annotations_values)
            else:
                value = annotation_key + ' : ' + annotations_values
            annotations.append(value)
        complete_nda_tier2_production_ref[k].update({'value':annotations})
        break
    break
complete_nda_tier2_production_ref